In [1]:
!pip -q install unsloth transformers datasets peft accelerate bitsandbytes scipy --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.2/372.2 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
# ============================================================
# Imports & basic config
# ============================================================
import os, json, glob, math, random, shutil, time
from pathlib import Path
from typing import List, Dict, Tuple
from collections import defaultdict

import numpy as np
import pandas as pd

import torch
from datasets import load_dataset
from huggingface_hub import snapshot_download
from scipy.spatial.distance import jensenshannon
from safetensors.torch import load_file as safe_load_file

from unsloth import FastLanguageModel
from peft import PeftModel
from peft.utils import set_peft_model_state_dict

from google.colab import userdata
from openai import OpenAI

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [6]:
# --------------------
# Repro & device
# --------------------
SEED = 3407
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ============================================================
# Base LM + tokenizer (also used as embedder + base baseline)
# ============================================================
BASE_MODEL_NAME = "unsloth/Meta-Llama-3.1-8B"
MAX_SEQ_LEN     = 2048
DTYPE           = None
LOAD_4BIT       = True

emb_model, base_tokenizer = FastLanguageModel.from_pretrained(
    model_name     = BASE_MODEL_NAME,
    max_seq_length = MAX_SEQ_LEN,
    dtype          = DTYPE,
    load_in_4bit   = LOAD_4BIT,
)
emb_model = FastLanguageModel.for_inference(emb_model).eval()

if base_tokenizer.pad_token is None:
    base_tokenizer.pad_token = base_tokenizer.eos_token
base_tokenizer.padding_side = "left"
SPECIAL_IDS = set(getattr(base_tokenizer, "all_special_ids", []) or [])

==((====))==  Unsloth 2025.12.4: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [7]:
# ============================================================
# Eval data: large-traversaal/urdu-instruct (test split)
# ============================================================
DATASET_NAME = "large-traversaal/urdu-instruct"
TARGET_LANG  = "ur"
MAX_EVAL     = 5   # change as needed

def load_eval_examples(limit: int):
    ds = load_dataset(DATASET_NAME, split="test")
    if limit is not None and len(ds) > limit:
        ds = ds.shuffle(seed=SEED).select(range(limit))
    return ds

eval_ds = load_eval_examples(MAX_EVAL)
print(f"[Data] Loaded {len(eval_ds)} Urdu instruct eval examples")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/7.45M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/147k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51686 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1084 [00:00<?, ? examples/s]

[Data] Loaded 5 Urdu instruct eval examples


In [8]:
# ============================================================
# Adapters on HF
# ============================================================
ADAPTERS = {
    "ur": "kanwal-mehreen18/urdu_lora_adapter",
    "tr": "anniew666/tr_lora_adapter",
    "pa": "anniew666/pa_lora_adapter",
    "ms": "anniew666/ms_lora_adapter",
    "fa": "anniew666/fa_lora_adapter",
    "bn": "anniew666/bn_lora_adapter",
    "ar": "anniew666/ar_lora_adapter",
}
ALL_LANGS = list(ADAPTERS.keys())

TASK_ADAPTER_REPO = "anniew666/urdu_task_adapter"

# Fixed baseline weights inside *language* group only (no more task vs language weighting)
LANG_FIXED_WEIGHTS  = (0.70, 0.15, 0.15)   # target, partner1, partner2

In [9]:
# ============================================================
# Stored statistics: BPE + centroids
# ============================================================
ART_DIR  = Path("/content/drive/MyDrive/Adaptive AI Project/Code Files/Lora Adapter Statistics/Statistics Files")
BPE_TOPK = 5000

def load_bpe_json(lang: str) -> Dict:
    p = ART_DIR / f"{lang}.bpe.top{BPE_TOPK}.json"
    with open(p, "r", encoding="utf-8") as f:
        obj = json.load(f)
    token_ids = np.array(obj["token_ids"], dtype=np.int64)
    probs     = np.array(obj["probs"], dtype=np.float64)
    probs     = probs / probs.sum() if probs.sum() > 0 else np.ones_like(probs)/len(probs)
    return {"token_ids": token_ids, "probs": probs}

def load_embed_json(lang: str) -> Dict:
    p = ART_DIR / f"{lang}.embed.json"
    with open(p, "r", encoding="utf-8") as f:
        obj = json.load(f)
    c = np.array(obj["centroid"], dtype=np.float32)
    c = c / (np.linalg.norm(c) + 1e-12)
    return {"centroid": c, "dim": int(obj["dim"])}

bpe_refs = {lg: load_bpe_json(lg) for lg in ALL_LANGS}
emb_refs = {lg: load_embed_json(lg) for lg in ALL_LANGS}
EMBED_DIM = emb_refs[TARGET_LANG]["dim"]
print(f"[Stats] Loaded BPE + centroids. EMBED_DIM={EMBED_DIM}")

[Stats] Loaded BPE + centroids. EMBED_DIM=4096


In [10]:
# ============================================================
# Embedding + BPE similarity helpers
# ============================================================
MAX_TOKENIZE_LEN = 512

@torch.no_grad()
def embed_one(text: str) -> np.ndarray:
    enc = base_tokenizer(
        [text],
        padding=True,
        truncation=True,
        max_length=MAX_TOKENIZE_LEN,
        return_tensors="pt",
    )
    enc = {k: v.to(DEVICE) for k, v in enc.items()}
    out = emb_model(**enc, output_hidden_states=True)
    last = out.hidden_states[-1]
    mask = enc["attention_mask"].unsqueeze(-1).type_as(last)
    pooled = (last * mask).sum(dim=1) / mask.sum(dim=1).clamp(min=1e-6)
    pooled = torch.nn.functional.normalize(pooled, p=2, dim=1)
    pooled = pooled.to(torch.float32)
    return pooled[0].detach().cpu().numpy()

def empirical_bpe_histogram(text: str, support_tokens: np.ndarray) -> np.ndarray:
    ids = base_tokenizer(text, add_special_tokens=False)["input_ids"]
    ids = [i for i in ids if i not in SPECIAL_IDS]
    if not ids:
        return np.ones(len(support_tokens), dtype=np.float64) / len(support_tokens)
    counts = np.zeros(len(support_tokens), dtype=np.float64)
    idx = {int(t): j for j, t in enumerate(support_tokens.tolist())}
    for tid in ids:
        j = idx.get(int(tid))
        if j is not None:
            counts[j] += 1.0
    if counts.sum() == 0:
        counts += 1e-8
    return counts / counts.sum()

def bpe_sim_jsd(p_input: np.ndarray, p_ref: np.ndarray) -> float:
    d = jensenshannon(p_input, p_ref, base=2.0)
    return 1.0 - float(d)

def cos_sim(a: np.ndarray, b: np.ndarray) -> float:
    return float(np.dot(a, b) / (np.linalg.norm(a)*np.linalg.norm(b) + 1e-12))

def softmax_weights(scores: List[float], temp: float = 0.07) -> List[float]:
    x = np.array(scores, dtype=np.float64) / max(1e-8, temp)
    x = x - x.max()
    w = np.exp(x)
    return (w / w.sum()).tolist()


In [11]:
SIM_SCHEMES = {
    "bpe_only":   (1.0,  0.0),
    "emb_only":   (0.0,  1.0),
    "mix_25_75":  (0.25, 0.75),
    "mix_50_50":  (0.50, 0.50),
    "mix_75_25":  (0.75, 0.25),
}

TOP_N_PARTNERS   = 2
INCLUDE_TARGET   = True
MIN_ACCEPT_SIM   = 0.0
SOFTMAX_TEMP     = 0.20
AGREEMENT_GUARD  = 0.10

def compute_per_lang_sims(text: str, evec: np.ndarray) -> List[Dict]:
    per_lang = []
    for lg in ALL_LANGS:
        ref_b = bpe_refs[lg]
        ref_e = emb_refs[lg]
        p_in  = empirical_bpe_histogram(text, ref_b["token_ids"])
        b_sim = bpe_sim_jsd(p_in, ref_b["probs"])
        e_sim = cos_sim(evec, ref_e["centroid"])
        per_lang.append({"lang": lg, "bpe_sim": b_sim, "emb_sim": e_sim})
    return per_lang

def select_adapters(per_lang: List[Dict], scheme: str, target_lang: str):
    alpha, beta = SIM_SCHEMES[scheme]

    scored = []
    for r in per_lang:
        b, e = r["bpe_sim"], r["emb_sim"]
        if alpha > 0 and beta > 0 and abs(b - e) > AGREEMENT_GUARD:
            s = (alpha*b + beta*e) * 0.9
        else:
            s = alpha*b + beta*e
        scored.append({**r, "scheme_score": float(max(0.0, min(1.0, s)))})

    ranked = sorted(scored, key=lambda x: x["scheme_score"], reverse=True)
    top_score = ranked[0]["scheme_score"]

    chosen = []
    if INCLUDE_TARGET and any(x["lang"] == target_lang for x in ranked):
        chosen.append(next(x for x in ranked if x["lang"] == target_lang))

    need = (1 + TOP_N_PARTNERS) if (INCLUDE_TARGET and chosen) else TOP_N_PARTNERS
    for r in ranked:
        if INCLUDE_TARGET and r["lang"] == target_lang:
            continue
        if len(chosen) >= need:
            break
        chosen.append(r)

    safety = False
    if top_score < MIN_ACCEPT_SIM:
        safety = True
        chosen = [next(x for x in ranked if x["lang"] == target_lang)] if (INCLUDE_TARGET and chosen) else [ranked[0]]

    weights = softmax_weights([c["scheme_score"] for c in chosen], temp=SOFTMAX_TEMP)
    s = sum(weights)
    weights = [w/s for w in weights]
    return chosen, weights, top_score, safety

In [12]:
# ============================================================
# Adapter caching + merge (IN-MEMORY ONLY)
# ============================================================
ADAPTER_CACHE = {}

def _download_adapter(repo: str):
    if repo in ADAPTER_CACHE:
        return ADAPTER_CACHE[repo]

    local = snapshot_download(repo)
    cfg_path = Path(local) / "adapter_config.json"
    with open(cfg_path, "r") as f:
        cfg = json.load(f)

    p_bin = Path(local) / "adapter_model.bin"
    if p_bin.exists():
        sd = torch.load(str(p_bin), map_location="cpu")
    else:
        p_st = Path(local) / "adapter_model.safetensors"
        if p_st.exists():
            sd = safe_load_file(str(p_st), device="cpu")
        else:
            merged = {}
            for sf in sorted(glob.glob(str(Path(local) / "*.safetensors"))):
                merged.update(safe_load_file(sf, device="cpu"))
            sd = merged

    ADAPTER_CACHE[repo] = {"cfg": cfg, "sd": sd}
    return ADAPTER_CACHE[repo]

def _is_lora_key(k: str) -> bool:
    kk = k.lower()
    return ("lora_" in kk) or ("loraa" in kk) or ("lorab" in kk)

def soft_merge_three_sd(primary_repo: str, p1_repo: str, p2_repo: str,
                        w0: float, w1: float, w2: float) -> Dict[str, torch.Tensor]:
    """Three-way LoRA merge, returns merged state_dict (no disk writes)."""
    prim = _download_adapter(primary_repo)
    p1   = _download_adapter(p1_repo)
    p2   = _download_adapter(p2_repo)

    sds = [prim["sd"], p1["sd"], p2["sd"]]
    merged = {}

    all_keys = set().union(*[sd.keys() for sd in sds])
    for k in all_keys:
        if not _is_lora_key(k):
            continue
        tens = []
        base = None
        for sd in sds:
            if k in sd and isinstance(sd[k], torch.Tensor):
                tens.append(sd[k])
                base = sd[k] if base is None else base
            else:
                tens.append(torch.zeros_like(base) if base is not None else None)
        if any(t is None for t in tens):
            continue
        merged[k] = w0*tens[0] + w1*tens[1] + w2*tens[2]

    # Copy non-LoRA keys from primary (modules_to_save, etc.)
    for k, v in prim["sd"].items():
        if not _is_lora_key(k):
            merged[k] = v

    return merged

def load_adapter_from_repo(adapter_repo: str):
    m, _ = FastLanguageModel.from_pretrained(
        model_name     = BASE_MODEL_NAME,
        max_seq_length = MAX_SEQ_LEN,
        dtype          = DTYPE,
        load_in_4bit   = LOAD_4BIT,
    )
    try:
        m.config.use_cache = False
    except:
        pass
    m = PeftModel.from_pretrained(m, adapter_repo)
    try:
        m.set_adapter(m.active_adapter)
    except:
        if hasattr(m, "peft_config") and len(m.peft_config):
            m.set_adapter(list(m.peft_config.keys())[0])
    return m.to(DEVICE).eval()

In [13]:
# ============================================================
# Build TASK_BASE_MODEL = base model with task adapter MERGED
# ============================================================
def build_task_merged_base():
    """
    Load the base Meta-Llama model, attach the Urdu task adapter,
    then MERGE the LoRA weights into the base weights (no weighting).
    """
    base, _ = FastLanguageModel.from_pretrained(
        model_name     = BASE_MODEL_NAME,
        max_seq_length = MAX_SEQ_LEN,
        dtype          = DTYPE,
        load_in_4bit   = LOAD_4BIT,
    )
    task_model = PeftModel.from_pretrained(base, TASK_ADAPTER_REPO)
    try:
        task_model.set_adapter(task_model.active_adapter)
    except:
        if hasattr(task_model, "peft_config") and len(task_model.peft_config):
            task_model.set_adapter(list(task_model.peft_config.keys())[0])

    task_merged = task_model.merge_and_unload()
    try:
        task_merged.config.use_cache = False
    except:
        pass

    return task_merged.eval()

In [14]:
TASK_BASE_MODEL = build_task_merged_base()
print("[Init] Built TASK_BASE_MODEL (base + task adapter merged).")

==((====))==  Unsloth 2025.12.4: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


adapter_config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:1222: UserWarning: Model has `tie_word_embeddings=True` and a tied layer is part of the adapter, but `ensure_weight_tying` is not set to True. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. Check the discussion here: https://github.com/huggingface/peft/issues/2777
  warnings.warn(msg)


adapter_model.safetensors:   0%|          | 0.00/2.77G [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


[Init] Built TASK_BASE_MODEL (base + task adapter merged).


In [15]:
# ============================================================
# Build language group sd and task+language models
# ============================================================
import copy
LANG_STRENGTH = 2.0

def build_language_group_sd(lang_ids: List[str], lang_weights: List[float]) -> Dict[str, torch.Tensor]:
    """
    Build a merged language LoRA state_dict from given language IDs and weights (sum ~1).
    Uses soft_merge_three_sd under the hood (up to 3 languages).

    This returns ONLY the adapter (LoRA/modules_to_save) weights.
    """
    assert len(lang_ids) == len(lang_weights) and len(lang_ids) >= 1

    # Normalize weights
    s = sum(lang_weights) or 1.0
    w_norm = [w / s for w in lang_weights]

    # If only one language, just return its adapter sd
    if len(lang_ids) == 1:
        repo = ADAPTERS[lang_ids[0]]
        return _download_adapter(repo)["sd"]

    # Up to 3 languages → pad & use soft_merge_three_sd
    ids = list(lang_ids)
    ws  = list(w_norm)
    while len(ids) < 3:
        ids.append(ids[-1])
        ws.append(0.0)
    ids = ids[:3]
    ws  = ws[:3]

    prim_repo = ADAPTERS[ids[0]]
    p1_repo   = ADAPTERS[ids[1]]
    p2_repo   = ADAPTERS[ids[2]]

    return soft_merge_three_sd(prim_repo, p1_repo, p2_repo, ws[0], ws[1], ws[2])

def build_task_plus_lang_model(lang_ids: List[str], lang_weights: List[float]):
    """
    Build a PEFT model corresponding to:

        final_adapter = task_adapter  (+ language_group_merge)

    i.e., task adapter is treated as the "base" adapter (full weight),
    and we ADD the language LoRA deltas on top (no extra weighting of task).

    Steps:
    - Build merged language adapter state_dict (lang_sd)
    - Load task adapter state_dict (task_sd)
    - Combine them into final_sd
    - Load a fresh task adapter model and overwrite its adapter weights
      using final_sd via plain state_dict update (no set_peft_model_state_dict).
    """

    # 1) Language group adapter (LoRA only)
    lang_sd = build_language_group_sd(lang_ids, lang_weights)

    # 2) Task adapter sd (LoRA + modules_to_save)
    task = _download_adapter(TASK_ADAPTER_REPO)
    task_sd = task["sd"]

    # 3) Combine LoRA keys: task as base, add language deltas
    final_sd = {}
    all_keys = set(lang_sd.keys()) | set(task_sd.keys())
    for k in all_keys:
        if _is_lora_key(k):
            t = task_sd.get(k)
            l = lang_sd.get(k)

            if t is None and l is None:
                continue
            if t is None:
                # no task value, only language
                final_sd[k] = LANG_STRENGTH * l
            elif l is None:
                # no language value, only task
                final_sd[k] = t
            else:
                # task full weight + language delta
                final_sd[k] = t + LANG_STRENGTH * l
        else:
            # Non-LoRA keys: prefer task adapter (modules_to_save etc.)
            v = task_sd.get(k, lang_sd.get(k))
            final_sd[k] = v

    # 4) Instantiate a task adapter model and update its state_dict
    mdl = load_adapter_from_repo(TASK_ADAPTER_REPO)

    # Get its current PEFT state dict and overwrite overlapping keys
    sd = mdl.state_dict()
    for k, v in final_sd.items():
        if k in sd:
            sd[k] = v

    # Load back, allowing missing keys (we only care about adapter params)
    mdl.load_state_dict(sd, strict=False)
    mdl.eval()
    return mdl

In [16]:
# ============================================================
# Generation helpers (chat template, decode prompt+completion)
# ============================================================
SYSTEM_PROMPT_URDU = """
آپ ایک مددگار معاون ہیں جو صرف آخری جواب اردو میں لکھتا ہے۔

قواعد:
- صرف حتمی جواب لکھیں۔
- سوال کو دوبارہ مت دہرائیں۔
- "system"، "user"، "assistant" یا کسی بھی رول کا نام مت لکھیں۔
- تاریخ، ہدایات، یا میٹا ڈیٹا (جیسے Cutting Knowledge Date وغیرہ) شامل نہ کریں۔
- بے معنی حروف یا الفاظ کی لمبی تکرار (جیسے racatraca وغیرہ) ہرگز نہ لکھیں۔
- جواب مختصر، واضح اور درست اردو میں ہو۔
""".strip()


@torch.no_grad()
def generate_text(model, prompt: str, max_new_tokens: int = 128) -> str:
    """
    Generate ONLY the assistant answer in Urdu, without using chat_template.

    We manually build an instruction-style prompt:

        [SYSTEM_PROMPT_URDU]

        سوال:
        <user prompt>

        جواب:

    Then decode only the newly generated tokens after this prefix.
    """
    full_prompt = (
        SYSTEM_PROMPT_URDU
        + "\n\nسوال:\n"
        + prompt.strip()
        + "\n\nجواب:"
    )

    enc = base_tokenizer(
        full_prompt,
        return_tensors="pt",
        truncation=True,
        max_length=MAX_TOKENIZE_LEN,
    )
    enc = {k: v.to(DEVICE) for k, v in enc.items()}

    out_ids = model.generate(
        **enc,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        eos_token_id=base_tokenizer.eos_token_id,
    )

    # Only the continuation after the prompt
    gen_ids = out_ids[0, enc["input_ids"].shape[-1]:]

    text = base_tokenizer.decode(gen_ids, skip_special_tokens=True)
    return text.strip()

In [17]:
# ============================================================
# LLM-as-judge (GPT-4o)
# ============================================================
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

JUDGE_SYSTEM_PROMPT = """The following are two systems’ outputs. Please evaluate both on a ten-point scale (1–10) against the original prompt, where 10 is the highest score, and provide an explanation for the scores.
You must:
- Use the full 1–10 scale (1 = terrible, 10 = exceptional).
- Be conservative with high scores (10 should be rare).
- Penalize even small issues: repetition, minor factual drift, awkward Urdu, or formatting noise.

The evaluation criteria are:
- Relevance: Does the response directly and adequately address the user’s prompt?
- Correctness: Is the information provided accurate and factually correct?
- Clarity: Is the response well-structured and free from unnecessary repetition or verbosity while maintaining completeness?
- Formatting Issues: Does the response have a consistent structure and is it free from unnecessary elements or incorrect language characters?

Respond in JSON with the following keys:
- "system1_score": integer 1–10
- "system2_score": integer 1–10
- "explanation": short text explanation.
"""

def judge_with_gpt(prompt: str, ref_output: str, cand_output: str,
                   model_name: str = "gpt-4o"):
    eval_user = f"""### Prompt: {prompt}
### System1: {ref_output}
### System2: {cand_output}"""

    resp = client.chat.completions.create(
        model=model_name,
        temperature=0.0,
        messages=[
            {"role": "system", "content": JUDGE_SYSTEM_PROMPT},
            {"role": "user",   "content": eval_user},
        ],
        # If available in your OpenAI version, you can uncomment this:
        # response_format={"type": "json_object"},
    )

    raw_text = resp.choices[0].message.content

    cleaned = raw_text.strip()
    if cleaned.startswith("```"):
        cleaned = cleaned.strip("`")
        if cleaned.lower().startswith("json"):
            cleaned = cleaned[4:].lstrip()

    try:
        data = json.loads(cleaned)
        s1 = data.get("system1_score")
        s2 = data.get("system2_score")
        expl = data.get("explanation", "")
    except Exception:
        print("⚠️ Judge output not valid JSON, storing raw text. Preview:\n", raw_text[:400])
        s1 = None
        s2 = None
        expl = raw_text

    return s1, s2, expl

In [18]:
# ============================================================
# Per-input evaluation over systems
# ============================================================
RESULTS = []
t_start = time.time()

w_t, w_p1, w_p2 = LANG_FIXED_WEIGHTS  # 0.70, 0.15, 0.15

for i, row in enumerate(eval_ds):
    prompt   = str(row["instruction"])
    ref_out  = str(row["output"])
    category = str(row.get("category", ""))

    preview = prompt[:160].replace("\n", " ")

    evec = embed_one(prompt)
    per_lang = compute_per_lang_sims(prompt, evec)

    # --------------------------------------------------------
    # 1) BASE: LLaMA only (no adapters)
    # --------------------------------------------------------
    base_out = generate_text(emb_model, prompt)
    s1, s2, expl = judge_with_gpt(prompt, ref_out, base_out)

    RESULTS.append({
        "idx": i,
        "category": category,
        "prompt_preview": preview,
        "system": "base",
        "sim_mode": "none",
        "alpha_bpe": None, "beta_emb": None,
        "choice_1_lang": None, "choice_1_weight": None,
        "choice_2_lang": None, "choice_2_weight": None,
        "choice_3_lang": None, "choice_3_weight": None,
        "top_score": None,
        "safety_fallback": False,
        "ref_output": ref_out,
        "system_output": base_out,
        "judge_system1_score": s1,
        "judge_system2_score": s2,
        "judge_explanation": expl,
    })

    # --------------------------------------------------------
    # 2) TASK ONLY: base + task adapter MERGED (TASK_BASE_MODEL)
    # --------------------------------------------------------
    task_out = generate_text(TASK_BASE_MODEL, prompt)
    s1, s2, expl = judge_with_gpt(prompt, ref_out, task_out)

    RESULTS.append({
        "idx": i,
        "category": category,
        "prompt_preview": preview,
        "system": "task_only",
        "sim_mode": "none",
        "alpha_bpe": None, "beta_emb": None,
        "choice_1_lang": None, "choice_1_weight": None,
        "choice_2_lang": None, "choice_2_weight": None,
        "choice_3_lang": None, "choice_3_weight": None,
        "top_score": None,
        "safety_fallback": False,
        "ref_output": ref_out,
        "system_output": task_out,
        "judge_system1_score": s1,
        "judge_system2_score": s2,
        "judge_explanation": expl,
    })

    # --------------------------------------------------------
    # 3) TASK + SINGLE LANGUAGE (Urdu only)
    # --------------------------------------------------------
    lang_ids_single = [TARGET_LANG]
    lang_w_single   = [1.0]

    model_single = build_task_plus_lang_model(lang_ids_single, lang_w_single)
    single_out = generate_text(model_single, prompt)
    del model_single; torch.cuda.empty_cache()

    s1, s2, expl = judge_with_gpt(prompt, ref_out, single_out)

    RESULTS.append({
        "idx": i,
        "category": category,
        "prompt_preview": preview,
        "system": "task+single_lang",
        "sim_mode": "single",
        "alpha_bpe": None, "beta_emb": None,
        "choice_1_lang": TARGET_LANG, "choice_1_weight": 1.0,
        "choice_2_lang": None, "choice_2_weight": None,
        "choice_3_lang": None, "choice_3_weight": None,
        "top_score": None,
        "safety_fallback": False,
        "ref_output": ref_out,
        "system_output": single_out,
        "judge_system1_score": s1,
        "judge_system2_score": s2,
        "judge_explanation": expl,
    })

    # --------------------------------------------------------
    # 4) TASK + FIXED LANGUAGE MIX (0.70, 0.15, 0.15 inside lang group)
    # --------------------------------------------------------
    chosen_fixed, _, top_score_fixed, safety_fixed = select_adapters(
        per_lang, scheme="mix_50_50", target_lang=TARGET_LANG
    )
    langs_fixed = [c["lang"] for c in chosen_fixed if c["lang"] != TARGET_LANG]

    if len(langs_fixed) == 0:
        lang_ids_fixed = [TARGET_LANG]
        lang_w_fixed   = [1.0]
        fp1 = fp2 = None
    else:
        if len(langs_fixed) == 1:
            fp1 = fp2 = langs_fixed[0]
        else:
            fp1, fp2 = langs_fixed[:2]

        lang_ids_fixed = [TARGET_LANG, fp1, fp2]
        lang_w_fixed   = [w_t, w_p1, w_p2]

    model_fixed = build_task_plus_lang_model(lang_ids_fixed, lang_w_fixed)
    fixed_out = generate_text(model_fixed, prompt)
    del model_fixed; torch.cuda.empty_cache()

    s1, s2, expl = judge_with_gpt(prompt, ref_out, fixed_out)

    RESULTS.append({
        "idx": i,
        "category": category,
        "prompt_preview": preview,
        "system": f"task+fixed_mix50:{TARGET_LANG}+{lang_ids_fixed}",
        "sim_mode": "fixed_mix_50_50",
        "alpha_bpe": 0.50, "beta_emb": 0.50,
        "choice_1_lang": TARGET_LANG,
        "choice_1_weight": lang_w_fixed[0],
        "choice_2_lang": fp1 if len(lang_ids_fixed) > 1 else None,
        "choice_2_weight": lang_w_fixed[1] if len(lang_ids_fixed) > 1 else None,
        "choice_3_lang": fp2 if len(lang_ids_fixed) > 2 else None,
        "choice_3_weight": lang_w_fixed[2] if len(lang_ids_fixed) > 2 else None,
        "top_score": float(top_score_fixed) if len(langs_fixed) else None,
        "safety_fallback": bool(safety_fixed),
        "ref_output": ref_out,
        "system_output": fixed_out,
        "judge_system1_score": s1,
        "judge_system2_score": s2,
        "judge_explanation": expl,
    })

    # --------------------------------------------------------
    # 5) TASK + ADAPTIVE LANGUAGE MIX (per similarity scheme)
    # --------------------------------------------------------
    for scheme, (alpha, beta) in SIM_SCHEMES.items():
        chosen, weights, top_score, safety = select_adapters(per_lang, scheme, TARGET_LANG)
        langs = [c["lang"] for c in chosen]

        model_adapt = build_task_plus_lang_model(langs, weights)
        adapt_out = generate_text(model_adapt, prompt)
        del model_adapt; torch.cuda.empty_cache()

        s1, s2, expl = judge_with_gpt(prompt, ref_out, adapt_out)

        RESULTS.append({
            "idx": i,
            "category": category,
            "prompt_preview": preview,
            "system": f"task+adaptive:{scheme}",
            "sim_mode": scheme,
            "alpha_bpe": alpha, "beta_emb": beta,
            "choice_1_lang": langs[0] if len(langs) > 0 else None,
            "choice_1_weight": weights[0] if len(weights) > 0 else None,
            "choice_2_lang": langs[1] if len(langs) > 1 else None,
            "choice_2_weight": weights[1] if len(weights) > 1 else None,
            "choice_3_lang": langs[2] if len(langs) > 2 else None,
            "choice_3_weight": weights[2] if len(weights) > 2 else None,
            "top_score": float(top_score),
            "safety_fallback": bool(safety),
            "ref_output": ref_out,
            "system_output": adapt_out,
            "judge_system1_score": s1,
            "judge_system2_score": s2,
            "judge_explanation": expl,
        })

    if (i + 1) % 5 == 0:
        elapsed = (time.time() - t_start) / 60
        print(f"[{i+1}/{len(eval_ds)}] examples | elapsed ~{elapsed:.1f} min")

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

adapter_config.json: 0.00B [00:00, ?B/s]

README.md:   0%|          | 0.00/613 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/2.77G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/606 [00:00<?, ?B/s]

==((====))==  Unsloth 2025.12.4: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
==((====))==  Unsloth 2025.12.4: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
==((====))==  Unsloth 2025.12.4: Fast Llama patching. Transfor

In [19]:
# ============================================================
# Save + summary
# ============================================================
df = pd.DataFrame(RESULTS)

# Compute relative score: system2 / system1 * 100
df["judge_rel_score_pct"] = np.where(
    (df["judge_system1_score"].notna()) &
    (df["judge_system1_score"] != 0),
    df["judge_system2_score"].astype(float) / df["judge_system1_score"].astype(float) * 100.0,
    np.nan,
)

out_csv = "/content/drive/MyDrive/Adaptive AI Project/Code Files/Results/ur_urdu_instruct_llm_judge_eval_taskmerged-langstrength2.csv"
df.to_csv(out_csv, index=False)
print("\n✅ Saved LLM-judge results to Google Drive:", out_csv)

summary = (
    df.groupby("system")["judge_rel_score_pct"]
      .mean()
      .reset_index()
      .sort_values("judge_rel_score_pct", ascending=False)
)

print("\n=== Avg relative score (% of reference) per system ===")
display(summary)

df


✅ Saved LLM-judge results to Google Drive: /content/drive/MyDrive/Adaptive AI Project/Code Files/Results/ur_urdu_instruct_llm_judge_eval_taskmerged-langstrength2.csv

=== Avg relative score (% of reference) per system ===


,system,judge_rel_score_pct
2,task+adaptive:emb_only,103.492063
7,task+single_lang,103.492063
3,task+adaptive:mix_25_75,103.492063
4,task+adaptive:mix_50_50,103.492063
5,task+adaptive:mix_75_25,103.492063
6,"task+fixed_mix50:ur+['ur', 'fa', 'ar']",103.492063
1,task+adaptive:bpe_only,102.500000
0,base,14.444444
8,task_only,14.166667


,idx,category,prompt_preview,system,sim_mode,alpha_bpe,beta_emb,choice_1_lang,choice_1_weight,choice_2_lang,...,choice_3_lang,choice_3_weight,top_score,safety_fallback,ref_output,system_output,judge_system1_score,judge_system2_score,judge_explanation,judge_rel_score_pct
0,0,qa,ٹی بی کو کس نظام سے منسلک بیماری کہا جاتا ہے؟,base,none,NaN,NaN,None,NaN,None,...,None,NaN,NaN,False,ٹی بی کو نظام تنفس سے منسلک بیماری کہا جاتا ہے۔,ٹی بی کو کس نظام سے منسلک بیماری کہا جاتا ہے؟\...,9,1,System1 provides a direct and correct answer t...,11.111111
1,0,qa,ٹی بی کو کس نظام سے منسلک بیماری کہا جاتا ہے؟,task_only,none,NaN,NaN,None,NaN,None,...,None,NaN,NaN,False,ٹی بی کو نظام تنفس سے منسلک بیماری کہا جاتا ہے۔,ٹی بی کو کس نظام سے منسلک بیماری کہا جاتا ہے؟\...,9,2,System1 correctly identifies that TB (Tubercul...,22.222222
2,0,qa,ٹی بی کو کس نظام سے منسلک بیماری کہا جاتا ہے؟,task+single_lang,single,NaN,NaN,ur,1.000000,None,...,None,NaN,NaN,False,ٹی بی کو نظام تنفس سے منسلک بیماری کہا جاتا ہے۔,ٹی بی کو پھیپھڑوں کے نظام سے منسلک بیماری کہا ...,9,9,Both systems provide accurate and relevant res...,100.000000
3,0,qa,ٹی بی کو کس نظام سے منسلک بیماری کہا جاتا ہے؟,"task+fixed_mix50:ur+['ur', 'fa', 'ar']",fixed_mix_50_50,0.50,0.50,ur,0.700000,fa,...,ar,0.150000,0.548250,False,ٹی بی کو نظام تنفس سے منسلک بیماری کہا جاتا ہے۔,ٹی بی کو پھیپھڑوں کے نظام سے منسلک بیماری کہا ...,9,9,Both responses are relevant and correctly iden...,100.000000
4,0,qa,ٹی بی کو کس نظام سے منسلک بیماری کہا جاتا ہے؟,task+adaptive:bpe_only,bpe_only,1.00,0.00,ur,0.616622,fa,...,ar,0.187593,0.288411,False,ٹی بی کو نظام تنفس سے منسلک بیماری کہا جاتا ہے۔,ٹی بی کو پھیپھڑوں کے نظام سے منسلک بیماری کہا ...,9,9,Both systems provided accurate and relevant re...,100.000000
5,0,qa,ٹی بی کو کس نظام سے منسلک بیماری کہا جاتا ہے؟,task+adaptive:emb_only,emb_only,0.00,1.00,ur,0.658195,fa,...,ar,0.107053,0.929922,False,ٹی بی کو نظام تنفس سے منسلک بیماری کہا جاتا ہے۔,ٹی بی کو پھیپھڑوں کے نظام سے منسلک بیماری کہا ...,9,9,Both responses are relevant and correctly iden...,100.000000
6,0,qa,ٹی بی کو کس نظام سے منسلک بیماری کہا جاتا ہے؟,task+adaptive:mix_25_75,mix_25_75,0.25,0.75,ur,0.621221,fa,...,ar,0.139495,0.692590,False,ٹی بی کو نظام تنفس سے منسلک بیماری کہا جاتا ہے۔,ٹی بی کو پھیپھڑوں کے نظام سے منسلک بیماری کہا ...,9,9,Both systems provide accurate and relevant res...,100.000000
7,0,qa,ٹی بی کو کس نظام سے منسلک بیماری کہا جاتا ہے؟,task+adaptive:mix_50_50,mix_50_50,0.50,0.50,ur,0.612085,fa,...,ar,0.158238,0.548250,False,ٹی بی کو نظام تنفس سے منسلک بیماری کہا جاتا ہے۔,ٹی بی کو پھیپھڑوں کے نظام سے منسلک بیماری کہا ...,9,9,Both responses are relevant and correctly iden...,100.000000
8,0,qa,ٹی بی کو کس نظام سے منسلک بیماری کہا جاتا ہے؟,task+adaptive:mix_75_25,mix_75_25,0.75,0.25,ur,0.601255,fa,...,ar,0.178957,0.403910,False,ٹی بی کو نظام تنفس سے منسلک بیماری کہا جاتا ہے۔,ٹی بی کو پھیپھڑوں کے نظام سے منسلک بیماری کہا ...,9,9,Both systems provide accurate and relevant res...,100.000000
9,1,classification,درج ذیل بیان کو 'روزمرہ کی زندگی'، 'تعلیم'، یا...,base,none,NaN,NaN,None,NaN,None,...,None,NaN,NaN,False,روزمرہ کی زندگی,,8,1,System1 correctly categorizes the statement un...,12.500000
